In [85]:
! pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [86]:
# PyTorch
from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler
import torch.nn as nn
from torchmetrics import ConfusionMatrix
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Data science tools
import numpy as np
import pandas as pd
import os

# Image manipulations
from PIL import Image
# Useful for examining network
from torchsummary import summary
# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

train_on_gpu = cuda.is_available()

%matplotlib inline

In [87]:
PathToDF = "/content/drive/MyDrive/Models1.csv"
DataSetName= "Mel128_200_M_12088"
RandomState= 42

# Func

In [88]:
def Save_To_CSV(PathToDF, df):
        if os.path.exists(PathToDF):
            df.to_csv(PathToDF, mode='a', header=False)
        else :
            df.to_csv(PathToDF)

In [89]:
def test(model, test_loader, criterion, CM=False):
  with torch.no_grad():
    model.eval()
    test_acc = 0
    preidcted = np.array([])
    ALL_Labels= np.array([])

    confusionMatrix= np.zeros((7,7))
    for images, labels in test_loader:

      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      
      _, pred = torch.max(outputs, dim=1)
      correct_tensor = pred.eq(labels.data.view_as(pred))
      accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
      test_acc += accuracy.item() * images.size(0)
      if CM:
        preidcted =np.append(preidcted ,pred.to("cpu").numpy())
        ALL_Labels =np.append(ALL_Labels ,labels.to("cpu").numpy())
    

    test_acc = test_acc / len(test_loader.dataset)

    if CM ==True:
      confmat = ConfusionMatrix(num_classes=7)
      confusionMatrix= np.array(confmat(torch.tensor(preidcted.astype("int32")), torch.tensor(ALL_Labels.astype("int32"))))

    return test_acc, confusionMatrix

# Load Data

In [90]:
torch.manual_seed(RandomState)

In [91]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
import warnings
from IPython.display import clear_output
warnings.filterwarnings('ignore')

In [93]:
transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])

dataset = datasets.ImageFolder(f'/content/drive/MyDrive/Data/{DataSetName}', transform=transform)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)


train_data = int(len(dataset)*0.8)
valid_data= int((len(dataset) - train_data)/2)
test_data = int(len(dataset) - train_data - valid_data)

train_set, val_set, test_set = torch.utils.data.random_split(dataset, [train_data, valid_data, test_data])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)



# Train pre trained VGG16

In [94]:
import torchvision.models as models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.vgg16(pretrained=True)

In [95]:
device

device(type='cuda')

In [96]:
def get_pretrained_model(model_name):

    if model_name == 'vgg16':
        model = models.vgg16(pretrained=True)

        # Freeze early layers
        # for param in model.parameters():
        #     param.requires_grad = False
        n_inputs = model.classifier[6].in_features

        # Add on classifier
        model.classifier[6] = nn.Sequential(
            nn.Linear(n_inputs, 4096), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(4096, 7), nn.LogSoftmax(dim=1))

    # Move to gpu and parallelize
    if train_on_gpu:
        model = model.to('cuda')

    return model

In [97]:
model = get_pretrained_model('vgg16')

In [98]:
# summary(model, input_size=(3, 224, 224), batch_size=32, device="cuda")

In [99]:
print(model.classifier[6])


Sequential(
  (0): Linear(in_features=4096, out_features=4096, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=4096, out_features=7, bias=True)
  (4): LogSoftmax(dim=1)
)


In [100]:
model.class_to_idx = dataset.class_to_idx
model.idx_to_class = {
    idx: class_
    for class_, idx in model.class_to_idx.items()
}
list(model.idx_to_class.items())[:7]

[(0, 'ar'), (1, 'de'), (2, 'en'), (3, 'es'), (4, 'fr'), (5, 'it'), (6, 'pt')]

In [101]:
def train(model,criterion,optimizer,train_loader,valid_loader,save_file_name,
          max_epochs_stop=3,n_epochs=20,print_every=1):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        save_file_name (str ending in '.pt'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        print_every (int): frequency of epochs to print training stats

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
        history (DataFrame): history of train and validation loss and accuracy
    """

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf
    valid_acc_max = 0

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = timer()

        # Training loop
        for ii, (data, target) in enumerate(train_loader):
            # Tensors to gpu
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()

            # Clear gradients
            optimizer.zero_grad()
            # Predicted outputs are log probabilities
            output = model(data)

            # Loss and backpropagation of gradients
            loss = criterion(output, target)
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()

                    # Forward pass
                    output = model(data)

                    # Validation loss
                    loss = criterion(output, target)
                    # Multiply average loss times the number of examples in batch
                    valid_loss += loss.item() * data.size(0)

                    # Calculate validation accuracy
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    valid_acc += accuracy.item() * data.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                # Calculate average accuracy
                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)

                history.append([train_loss, valid_loss, train_acc, valid_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                    print(
                        f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                    )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                # if valid_acc > valid_acc_max:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_acc_max= valid_acc
                    # valid_best_acc = valid_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc_max:.2f}%'
                        )
                        total_time = timer() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict
                        model.load_state_dict(torch.load(save_file_name))
                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_acc',
                                'valid_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    return model, history

## optim and loss

In [102]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005, weight_decay = 0.005, momentum = 0.9)  


In [ ]:
rd= np.random.randint(100000)

modelName= f"VGG_{DataSetName}_{rd}"

model, history = train(
    model,
    criterion,
    optimizer,
    train_loader,
    valid_loader,
    save_file_name=f"/content/drive/MyDrive/Pre_VGG_models/{modelName}.pt",
    max_epochs_stop=6,
    n_epochs=100,
    print_every=1)

Starting Training from Scratch.


Epoch: 0 	Training Loss: 1.3622 	Validation Loss: 1.0124
		Training Accuracy: 44.89%	 Validation Accuracy: 58.97%

Epoch: 1 	Training Loss: 0.9020 	Validation Loss: 0.7689
		Training Accuracy: 64.95%	 Validation Accuracy: 71.05%

Epoch: 2 	Training Loss: 0.7669 	Validation Loss: 0.6093
		Training Accuracy: 71.17%	 Validation Accuracy: 77.50%

Epoch: 3 	Training Loss: 0.6425 	Validation Loss: 0.6987
		Training Accuracy: 75.81%	 Validation Accuracy: 73.78%

Epoch: 4 	Training Loss: 0.5699 	Validation Loss: 0.6572
		Training Accuracy: 78.66%	 Validation Accuracy: 76.10%

Epoch: 5 	Training Loss: 0.4902 	Validation Loss: 0.8427
		Training Accuracy: 82.19%	 Validation Accuracy: 72.87%

Epoch: 6 	Training Loss: 0.4598 	Validation Loss: 0.4313
		Training Accuracy: 82.89%	 Validation Accuracy: 84.53%


# Test

In [ ]:
test_acc, CM_Test  = test(model.cuda(), test_loader, criterion, True)
print(f'The model has achieved an accuracy of {100 * test_acc:.2f}% on the test dataset')

train_acc,CM_Train = test(model.cuda(), train_loader, criterion, True)
print(f'The model has achieved an accuracy of {100 * train_acc:.2f}% on the trian dataset')

In [ ]:
best_Result = history[history["valid_acc"]==history["valid_acc"].max()]
Models_Results = {
      "Model_Path":[modelName],
      "DataSetName":[DataSetName],

      "train_acc_Best_Eb":[best_Result.train_acc.values[0]],
      "vali_acc_Best_Eb":[best_Result.valid_acc.values[0]],

      "train_acc_All":[train_acc],
      "test_acc_All":[test_acc],
      "CM_Test": [ str(np.array(CM_Test).reshape(7*7).tolist())],
      "CM_Train" :  [ str(np.array(CM_Train).reshape(7*7).tolist())]
}

In [ ]:
Save_To_CSV(PathToDF,pd.DataFrame.from_dict(Models_Results))

In [ ]:
CM_Test.sum()

In [ ]:
# target = torch.tensor([1, 1, 0, 0])
# preds = torch.tensor([0, 1, 0, 0])
# confmat = ConfusionMatrix(num_classes=2)
# o = confmat(preds, target)

In [ ]:
# gih = pd.read_csv("test2.csv",converters={'a': pd.eval})